In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time
import psutil
import csv
import os

In [13]:
# -------- Force CUDA --------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Enable GPU in notebook settings.")
device = torch.device("cuda")
print(f"Using GPU: {torch.cuda.get_device_name(device)}")

# -------- Dataset and Transform --------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # mean, std
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# -------- Model Definition --------
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 8 * 8, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = SimpleCNN().to(device)

# -------- Loss and Optimizer --------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)




Using GPU: Tesla T4


In [14]:
# -------- CSV Logging Setup --------
csv_file = 'pytorch_cnn_cuda.csv'
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "Epoch", "Image_Number", "Batch_Index", "Loss",
        "RAM_Usage_MB", "GPU_Alloc_MB", "Time_Per_Batch_s",
        "Train_Accuracy(%)", "Test_Accuracy(%)"
    ])

# -------- Training Loop --------
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    correct_train, total_train = 0, 0

    for batch_idx, (images, labels) in enumerate(trainloader):
        start = time.perf_counter()

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Training accuracy for this batch
        _, predicted = torch.max(outputs.data, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        end = time.perf_counter()

        # Metrics
        ram = psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)
        gpu_mem = torch.cuda.memory_allocated() / (1024 ** 2)
        elapsed = end - start
        image_number = batch_idx * images.size(0)

        # Log per-batch
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1, image_number, batch_idx, f"{loss.item():.4f}",
                f"{ram:.2f}", f"{gpu_mem:.2f}", f"{elapsed:.6f}", "", ""
            ])

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}] Image {image_number}, Batch {batch_idx}: Loss={loss.item():.4f}, RAM={ram:.2f}MB, GPU={gpu_mem:.2f}MB, Time={elapsed:.6f}s")

    # -------- Train Accuracy This Epoch --------
    train_accuracy = 100 * correct_train / total_train

    # -------- Test Accuracy This Epoch --------
    model.eval()
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)

    test_accuracy = 100 * correct_test / total_test
    print(f"[Epoch {epoch+1}] Train Accuracy: {train_accuracy:.2f}% | Test Accuracy: {test_accuracy:.2f}%")
    model.train()

    # -------- Log Epoch-Level Accuracy to CSV --------
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            epoch + 1, "END", "-", "-", "-", "-", "-", f"{train_accuracy:.2f}", f"{test_accuracy:.2f}"
        ])

print(f"Final GPU Memory Used: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")


[Epoch 1] Image 0, Batch 0: Loss=2.3081, RAM=1514.60MB, GPU=18.62MB, Time=0.003213s
[Epoch 1] Image 6400, Batch 100: Loss=2.2836, RAM=1514.73MB, GPU=18.62MB, Time=0.002523s
[Epoch 1] Image 12800, Batch 200: Loss=2.2200, RAM=1514.73MB, GPU=18.62MB, Time=0.002529s
[Epoch 1] Image 19200, Batch 300: Loss=2.1590, RAM=1514.73MB, GPU=18.62MB, Time=0.002546s
[Epoch 1] Image 25600, Batch 400: Loss=2.0753, RAM=1514.73MB, GPU=18.62MB, Time=0.002537s
[Epoch 1] Image 32000, Batch 500: Loss=2.1371, RAM=1514.73MB, GPU=18.62MB, Time=0.002512s
[Epoch 1] Image 38400, Batch 600: Loss=2.1322, RAM=1514.73MB, GPU=18.62MB, Time=0.002599s
[Epoch 1] Image 44800, Batch 700: Loss=1.8386, RAM=1514.73MB, GPU=18.62MB, Time=0.002604s
[Epoch 1] Train Accuracy: 23.71% | Test Accuracy: 32.62%
[Epoch 2] Image 0, Batch 0: Loss=1.9390, RAM=1515.23MB, GPU=18.62MB, Time=0.002873s
[Epoch 2] Image 6400, Batch 100: Loss=1.9281, RAM=1515.23MB, GPU=18.62MB, Time=0.002609s
[Epoch 2] Image 12800, Batch 200: Loss=1.8611, RAM=1515.2